## Instruções

* Rodar o notebook em ordem
* Arquivos necessários:
    * preprocessed_CETEN_v2.pkl: gerado pelo notebook 1_Data_Prep.ipynb
    * glove_s50.txt: Disponível em http://nilc.icmc.usp.br/embeddings
    
    

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import io
import re
from collections import Counter
import gc
import time
import pickle
import nltk
import math
from sklearn.model_selection import train_test_split

In [3]:
with open('preprocessed_CETEN_v2.pkl', 'rb') as input:
    phrases = pickle.load(input)

#### Carrega GloVe word embeddings

In [6]:
from gensim.models import KeyedVectors

In [7]:
glove = KeyedVectors.load_word2vec_format('./glove_s50.txt')

#### Divide a base em treino e teste

In [15]:
phrases_train , phrases_test = train_test_split(phrases,test_size=0.2, random_state=42)
#phrases_train , phrases_valid = train_test_split(phrases_train,test_size=0.2, random_state=42)

#### Converte qualquer palavra com contagem menor que 5 para RARE

In [16]:
min_threshold = 5

word_counts = Counter()
for phrase in phrases_train:
    for word in phrase:
        word_counts[word[0]]+=1
        
word_counts = {word: count for word, count in word_counts.items() if word_counts[word] >= min_threshold}

In [17]:
phrases_train_rare = []
RARE_WORD = '__RARE__'
for phrase in phrases_train:
    phrases_train_rare.append([(w[0] if word_counts.get(w[0]) else RARE_WORD,w[1]) for w in phrase])

#### Contagem de palavras na base de treino


In [18]:
BATCH_SIZE=1

In [19]:
SAMPLE=100001

In [20]:
tags_counter = Counter()
for s in phrases_train_rare[:SAMPLE]:
    for tk in s:
        tag = tk[1]
        if tags_counter[tag]:
            tags_counter[tag]=tags_counter[tag]+1
        else:
            tags_counter[tag] = 1

In [21]:
allowed_tags = tags_counter.keys()

In [22]:
word_freq = {}
for s in phrases_train_rare[:SAMPLE]:
    for tk in s:
        if word_freq.get(tk[0]) == None:
            word_freq[tk[0]] = Counter()
        word_freq[tk[0]][tk[1]] = word_freq[tk[0]][tk[1]] + 1

#### Definição da lista de TAGs

In [23]:
allowed_tags_pad = ['<PAD>']+list(allowed_tags)

In [24]:
allowed_tags_to_ix = { tag:i for i,tag in enumerate(allowed_tags_pad)}
ix_to_allowed_tags = { i:tag for i,tag in enumerate(allowed_tags_pad)}

#### Funções auxiliares

In [25]:
def collect0(lst):
    return list(map(lambda x: x[0], lst))
def collect1(lst):
    return list(map(lambda x: x[1], lst))
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


#### Definição da base de treino e de validação

In [26]:
training_data_phrases = [(collect0(p),collect1(p)) for p in phrases_train_rare[:SAMPLE] if collect0(p) != ['']]
valid_data_phrases = [(collect0(p),collect1(p)) for p in phrases_train_rare[SAMPLE:SAMPLE+10000] if collect0(p) != ['']]

#### Definição dos dicionários
* word_to_ix
* tag_to_ix
* ix_to_tag

In [27]:
training_data = np.array(training_data_phrases)
valid_data = np.array(valid_data_phrases)[2000:4000]
print("Number of phrases: ", len(training_data))
word_to_ix = {'<PAD>':0}
char_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
        
        for char in word:
            if char not in char_to_ix:
                char_to_ix[char] = len(char_to_ix)
print("Number of words: ", len(word_to_ix))
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}
tag_to_ix = allowed_tags_to_ix
ix_to_tag = ix_to_allowed_tags
print("Number of tags: ", len(allowed_tags_to_ix.keys()))

Number of phrases:  99997
Number of words:  59709
Number of tags:  17


#### Definição das dimensões da rede

In [30]:
EMBEDDING_DIM = 50
HIDDEN_DIM = 200

#### Salva dicionários: serão carregados para a etapa de avaliação

In [21]:
with open('model_lstm_dicts.pkl', 'wb') as f:
    pickle.dump({'word_to_ix':word_to_ix, 'tag_to_ix':tag_to_ix, 'ix_to_tag':ix_to_tag, 'BATCH_SIZE':1}, f, pickle.HIGHEST_PROTOCOL)

#### Inicializa embeddings weights com Glove50

In [32]:
matrix_len = len(word_to_ix)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(word_to_ix.keys()):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM, ))
print(words_found)        

44774


### Definição da rede LSTM

In [34]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [47]:
##BACKUP WORKING LSTM biredirectional with gloves100
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, word_to_ix, embedding_weights):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.tagset_size = tagset_size

        padding_idx = word_to_ix['<PAD>']
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.word_embeddings.weight.data.copy_(torch.from_numpy(embedding_weights))
        
        self.num_layers = 1

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, self.num_layers, batch_first=False, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(self.num_layers*2, BATCH_SIZE, self.hidden_dim),
                torch.zeros(self.num_layers*2, BATCH_SIZE, self.hidden_dim))
        #return torch.zeros(1, 1, self.hidden_dim)

    def forward(self, sentence, s_lengths, debug=False):
        batch_size, seq_len, = sentence.size()
        
        
        
        if debug: print("sentences input:", sentence.size())
        # 1. embed the input
        # Dim transformation: (batch_size, seq_len, 1) -> (batch_size, seq_len, embedding_dim)
        embeds = self.word_embeddings(sentence)
        if debug: print("embeds:",embeds.size())
        
        # 2. Run through RNN
        # TRICK 2 ********************************
        # Dim transformation: (batch_size, seq_len, embedding_dim) -> (batch_size, seq_len, nb_lstm_units)
        
        embeds = embeds.transpose(0,1)
        # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        lstm_input = torch.nn.utils.rnn.pack_padded_sequence(embeds, s_lengths, batch_first=False)        
        #lstm_input = embeds.view(seq_len, BATCH_SIZE, -1)
        #lstm_input = embeds.transpose(0,1)
        if debug: print("lstm_input:",lstm_input.data.size())
        
        if debug: print("hidden0 (ht):",self.hidden[0].size())
        if debug: print("hidden1 (hc):",self.hidden[1].size())    
        # now run through LSTM
        lstm_out, self.hidden = self.lstm(lstm_input, self.hidden)
        
        if debug: print("lstm_out:", lstm_out.data.size())
        if debug: print("hidden0 (ht):",self.hidden[0].size())
        if debug: print("hidden1 (hc):",self.hidden[1].size())    
        
        # undo the packing operation
        lstm_out, _ = torch.nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=False)
        #print("lstm_out:", lstm_out.data.size())
        
        
        # 3. Project to tag space
        # Dim transformation: (batch_size, seq_len, nb_lstm_units) -> (batch_size * seq_len, nb_lstm_units)

        # this one is a bit tricky as well. First we need to reshape the data so it goes into the linear layer
        #lstm_out = lstm_out.contiguous()
        #lstm_out = lstm_out.view(-1, lstm_out.shape[2])
        #lstm_out = lstm_out.view(seq_len, -1)
        
        ##OPTION1 
        #batch_first = lstm_out.transpose(0,1)
        #print("batch_first:", batch_first.size())
        #batch_first = batch_first.contiguous()
        #print("batch_first(contiguous):", batch_first.size())
        #linear_input = batch_first.view(BATCH_SIZE, -1)
        ### Para fazer isso, preciso setar o input da Linear ao inves de 200, para 200*N,
        #onde N é o tamanho da maior sentença da base toda
        
        
        
        ###OPTION2
        #linear_input = self.hidden[0]
        
        
        ##option3
        lstm_out = lstm_out.transpose(0,1)
        lstm_out = lstm_out.contiguous()
        if debug: print("lstm_out reshaped:", lstm_out.size())
        linear_input = lstm_out.view(-1, lstm_out.shape[2])
        if debug: print("linear_input:", linear_input.size())
        
        
        
        tag_space = self.hidden2tag(linear_input)
        if debug: print("hidden out :", tag_space.size())
        tag_scores = F.log_softmax(tag_space, dim=1)
        
        #tag_scores = tag_scores.view(BATCH_SIZE, seq_len, self.tagset_size)
        return tag_scores

#### Testa se o feed-forwaed está funcionando!

In [48]:
test_sentence = training_data[0][0]

In [49]:
BATCH_SIZE=1
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix), word_to_ix,weights_matrix)

#tag_scores = model(sentence_in, s_lengths)
inputs = prepare_batch_sequence([test_sentence]*BATCH_SIZE, word_to_ix)
tag_scores = model(inputs, [len(inputs[0])]*BATCH_SIZE, debug=True)
tag_scores.shape
#tag_scores

sentences input: torch.Size([1, 19])
embeds: torch.Size([1, 19, 50])
lstm_input: torch.Size([19, 50])
hidden0 (ht): torch.Size([2, 1, 200])
hidden1 (hc): torch.Size([2, 1, 200])
lstm_out: torch.Size([19, 400])
hidden0 (ht): torch.Size([2, 1, 200])
hidden1 (hc): torch.Size([2, 1, 200])
lstm_out reshaped: torch.Size([1, 19, 400])
linear_input: torch.Size([19, 400])
hidden out : torch.Size([19, 17])


torch.Size([19, 17])

#### Definição de funções auxiliares para treinamento

In [50]:
def prepare_batch_sequence(sentences, to_ix):
    s_lengths = [len(s) for s in sentences]

    # create an empty matrix with padding tokens
    pad_token = to_ix['<PAD>']
    longest_sent = max(s_lengths)
    batch_size = len(sentences)
    padded_sentences = np.ones((batch_size, longest_sent)) * pad_token
    # copy over the actual sequences
    for n, s_len in enumerate(s_lengths):
        sequence = sentences[n]
        idxs = [to_ix[w] for w in sequence[:s_len]]
        padded_sentences[n, 0:s_len] = idxs
    
    return torch.tensor(padded_sentences, dtype=torch.long)

def batch_idx_loader(data, shuffle=True):
    permutation = torch.randperm(len(data)) if shuffle else torch.tensor(range(len(data)))
    gen = (permutation[b:b+BATCH_SIZE] for b in range(0,len(data), BATCH_SIZE))
    return gen

def get_data_sorted(data, idx):
    data = np.array(data)
    idx = [idx] if len(idx) == 1 else idx
    
    sentences = [bat[0] for bat in data[idx]]
    tags =      [bat[1] for bat in data[idx]]

    mydict     = {idx:len(s) for idx,s in enumerate(sentences)}
    idx_sorted = [k for k in sorted(mydict, key=mydict.get, reverse=True)]

    sentences = np.array(sentences)[idx_sorted]
    tags      = np.array(tags)[idx_sorted]
    
    return sentences, tags

#### Treinamento...

In [374]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix), word_to_ix, weights_matrix)

tag_pad_token = tag_to_ix['<PAD>']
loss_function = nn.NLLLoss(ignore_index=tag_pad_token)

optimizer = optim.SGD(model.parameters(), lr=0.1)


since = time.time()
EPOCHS = 3
BATCH_SIZE=1
for epoch in range(EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data
    
    model.train()
    running_loss = 0.0
    running_corrects = 0
    running_examples = 0
    i=0
    
    
    
    SAMPLE_C = training_data.shape[0]
    SAMPLE_C = int(SAMPLE_C/BATCH_SIZE)*BATCH_SIZE
    dt=training_data[:SAMPLE_C]
    loader = batch_idx_loader(dt)
    for indices in loader:
        
        model.train()
        
        sentences, tags = get_data_sorted(dt, indices)
        
        i=i+1
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        #sentence_in = prepare_sequence(sentence, word_to_ix)
        sentence_in = prepare_batch_sequence(sentences,word_to_ix)
        #chars_in    = prepare_char_sequence(sentences,word_to_ix)
        
        #targets = prepare_sequence(tags, tag_to_ix)
        targets = prepare_batch_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        s_lengths = [len(s) for s in sentences]
        tag_scores = model(sentence_in, s_lengths)
        _, preds = torch.max(tag_scores, 1)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets.view(-1))
        #print("vai chamar o backward")
        loss.backward()
        optimizer.step()
        
        
        
        # print statistics
        running_corrects += (preds == targets.view(-1)).cpu().numpy().sum()
        running_loss += loss.item()
        running_examples += preds.shape[0]
        if i % 1000 == 999:    # print every 100 mini-batches
            torch.save(model.state_dict(), "model_ep"+str(epoch + 1)+"_bs1_state_dict.model")
            model.eval()
            v_running_loss = 0.0
            v_running_corrects = 0
            v_running_examples = 0
            v_i=0
            for v_sentence, v_tags in valid_data:
                v_i=v_i+1
                v_sentence_rare = [w if word_to_ix.get(w) != None else RARE_WORD for w in v_sentence]
                v_sentence_in = prepare_batch_sequence([v_sentence_rare]*BATCH_SIZE, word_to_ix)
                v_targets = prepare_batch_sequence([v_tags]*BATCH_SIZE, tag_to_ix)
                v_seq_len=len(v_sentence_in[0])
                v_tag_scores = model(v_sentence_in, [v_seq_len]*BATCH_SIZE)
                _, v_preds = torch.max(v_tag_scores, 1)
                v_preds = v_preds.view(BATCH_SIZE, v_seq_len)
                v_loss = loss_function(v_tag_scores, v_targets.view(-1))
                v_running_corrects += (v_preds[0] == v_targets[0]).cpu().numpy().sum()
                v_running_loss += v_loss.item()
                v_running_examples += v_preds.shape[1]
        
            #print(v_tag_scores.shape)
            #print(v_targets.shape)
            #print(v_preds.shape)
            #print(v_running_corrects)
            #print(v_running_examples)
            print('Train: [%d, %5d, %5d] loss: %.4f  acc:%.3f //// Valid  loss: %.4f  acc:%.3f' %
                  (epoch + 1,
                   i + 1,
                   (i+1)*BATCH_SIZE,
                   running_loss / 100,
                   running_corrects /running_examples*100,
                   v_running_loss / valid_data.shape[0],
                   v_running_corrects /v_running_examples*100))
            running_loss = 0.0
            running_corrects = 0
            running_examples = 0

time_elapsed = time.time() - since
print('Training complete ({} epochs)in {:.0f}m {:.0f}s'.format(
    EPOCHS,time_elapsed // 60, time_elapsed % 60))




Train: [1,  1000,  1000] loss: 12.400  acc:64.33 //// Valid  loss: 0.910  acc:74.57
Train: [1,  2000,  2000] loss: 6.942  acc:79.10 //// Valid  loss: 0.713  acc:81.71
Train: [1,  3000,  3000] loss: 5.883  acc:81.83 //// Valid  loss: 0.604  acc:84.27
Train: [1,  4000,  4000] loss: 5.102  acc:84.64 //// Valid  loss: 0.572  acc:85.65
Train: [1,  5000,  5000] loss: 4.908  acc:85.90 //// Valid  loss: 0.531  acc:86.77
Train: [1,  6000,  6000] loss: 4.399  acc:87.35 //// Valid  loss: 0.549  acc:85.78
Train: [1,  7000,  7000] loss: 4.024  acc:87.82 //// Valid  loss: 0.503  acc:87.45
Train: [1,  8000,  8000] loss: 3.573  acc:89.37 //// Valid  loss: 0.521  acc:87.87
Train: [1,  9000,  9000] loss: 3.729  acc:89.06 //// Valid  loss: 0.445  acc:89.43
Train: [1, 10000, 10000] loss: 3.340  acc:90.12 //// Valid  loss: 0.456  acc:89.51
Train: [1, 11000, 11000] loss: 3.392  acc:91.00 //// Valid  loss: 0.445  acc:89.48
Train: [1, 12000, 12000] loss: 3.254  acc:90.64 //// Valid  loss: 0.388  acc:91.03
Tra

Train: [2,  1000,  1000] loss: 1.162  acc:96.52 //// Valid  loss: 0.246  acc:95.67
Train: [2,  2000,  2000] loss: 1.421  acc:96.20 //// Valid  loss: 0.220  acc:95.65
Train: [2,  3000,  3000] loss: 1.261  acc:96.25 //// Valid  loss: 0.243  acc:95.67
Train: [2,  4000,  4000] loss: 1.337  acc:96.23 //// Valid  loss: 0.233  acc:95.66
Train: [2,  5000,  5000] loss: 1.193  acc:96.57 //// Valid  loss: 0.220  acc:95.75
Train: [2,  6000,  6000] loss: 1.316  acc:96.40 //// Valid  loss: 0.221  acc:95.70
Train: [2,  7000,  7000] loss: 1.233  acc:96.39 //// Valid  loss: 0.224  acc:95.79
Train: [2,  8000,  8000] loss: 1.375  acc:96.51 //// Valid  loss: 0.229  acc:95.78
Train: [2,  9000,  9000] loss: 1.256  acc:96.26 //// Valid  loss: 0.238  acc:95.74
Train: [2, 10000, 10000] loss: 1.248  acc:96.21 //// Valid  loss: 0.227  acc:95.32
Train: [2, 11000, 11000] loss: 1.123  acc:96.31 //// Valid  loss: 0.233  acc:95.76
Train: [2, 12000, 12000] loss: 1.126  acc:96.58 //// Valid  loss: 0.214  acc:95.84
Trai

Train: [3,  1000,  1000] loss: 0.834  acc:97.18 //// Valid  loss: 0.199  acc:96.34
Train: [3,  2000,  2000] loss: 0.882  acc:97.27 //// Valid  loss: 0.197  acc:96.64
Train: [3,  3000,  3000] loss: 0.810  acc:97.29 //// Valid  loss: 0.200  acc:96.52
Train: [3,  4000,  4000] loss: 0.820  acc:97.38 //// Valid  loss: 0.212  acc:96.47
Train: [3,  5000,  5000] loss: 1.000  acc:97.11 //// Valid  loss: 0.197  acc:96.53
Train: [3,  6000,  6000] loss: 0.835  acc:97.13 //// Valid  loss: 0.194  acc:96.49
Train: [3,  7000,  7000] loss: 0.948  acc:97.06 //// Valid  loss: 0.234  acc:96.19
Train: [3,  8000,  8000] loss: 0.815  acc:97.27 //// Valid  loss: 0.186  acc:96.61
Train: [3,  9000,  9000] loss: 0.787  acc:97.56 //// Valid  loss: 0.208  acc:96.36
Train: [3, 10000, 10000] loss: 0.829  acc:97.20 //// Valid  loss: 0.201  acc:96.49
Train: [3, 11000, 11000] loss: 0.749  acc:97.56 //// Valid  loss: 0.196  acc:96.66
Train: [3, 12000, 12000] loss: 0.825  acc:97.28 //// Valid  loss: 0.224  acc:96.31
Trai

Training complete (3 epochs)in 59m 11s


#### Salva modelo para ser utilizado na avaliação

In [375]:
torch.save(model.state_dict(), "model_96acc_bs1_state_dict.model")

In [376]:
torch.save(model, "model_96acc_bs1_full.model")

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type LSTMTagger. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#### Testa performance do modelo

In [ ]:
def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_tag):
    return ix_to_tag[get_index_of_max(input)]

### Testa classificação em uma sentença de exemplo

In [365]:
test_sentence = training_data[0][0]
inputs = prepare_batch_sequence([test_sentence]*BATCH_SIZE, word_to_ix)
seq_len=len(inputs[0])
tag_scores = model(inputs, [seq_len]*BATCH_SIZE)
tag_scores = tag_scores.view(BATCH_SIZE, seq_len, len(tag_to_ix))
for i in range(len(test_sentence)):
    print('{}: {}'.format(test_sentence[i],get_max_prob_result(tag_scores[0][i].data.cpu().numpy(), ix_to_tag)))

ao=longo=de: NUM
décadas: N
$,: __PUNCT__
a: DET
limitada: V
capacidade: N
de: PRP
importar: V
de: PRP
o: DET
país: N
veio: V
preocupando: V
os: DET
formuladores: N
de: PRP
política: N
econômica: ADJ
$.: __PUNCT__


#### Avalia performance em base de validação

In [372]:
tdef=0
tloop1=0
tloop2=0
ttag=0
corretas_baseline = 0
corretas_rnn = 0
totais = 0
phrases_test_with_rare = []

testwith = 2000
testwith = int(testwith/BATCH_SIZE)*BATCH_SIZE

### Add RARE 
for s in phrases_test[:testwith]:
    phrases_test_with_rare.append([(tk[0],tk[1]) if word_to_ix.get(tk[0]) != None else (RARE_WORD,tk[1]) for tk in s])
    
###Define test dataset    
validating_data_phrases = [(collect0(p),collect1(p)) for p in phrases_test_with_rare]



### Score testdataset into preds variable
preds=[]
dt = validating_data_phrases
loader = batch_idx_loader(dt, shuffle=False)
for indices in loader:
    sentences, tags = get_data_sorted(dt, indices)
    
    sentence_in = prepare_batch_sequence(sentences,word_to_ix)
    targets = prepare_batch_sequence(tags, tag_to_ix)
    
    s_lengths = [len(s) for s in sentences]
    tag_scores = model(sentence_in, s_lengths)
    tag_scores = tag_scores.view(BATCH_SIZE, s_lengths[0], len(tag_to_ix))
    
    for batchline in range(tag_scores.shape[0]):
        pred = [get_max_prob_result(tag_scores[batchline][wordidx].data.cpu().numpy(), ix_to_tag) for wordidx in range(s_lengths[batchline])]
        preds.append([(word, golden, tk) for word, golden, tk in zip(sentences[batchline], tags[batchline], pred)])

for pred in preds:
    for word, tk_golden, tk_pred in pred:
        #print(tk_golden)
        totais+=1
        if tk_golden == word_freq[word].most_common(1)[0][0]:
            corretas_baseline+=1
        if tk_golden == tk_pred:
            corretas_rnn+=1
            
corretas_rnn, corretas_baseline, totais, corretas_rnn/totais*100, corretas_baseline/totais*100

(31186, 30520, 32526, 95.88021890180164, 93.83262620672693)